In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import librosa
from keras.preprocessing import image
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, GlobalMaxPooling2D, BatchNormalization, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
sys.path.append('/content/gdrive/MyDrive')

In [ ]:
def audio_to_mel(file_path, image_file):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    y, sr = librosa.load(file_path)
    ms = librosa.feature.melspectrogram(y=y, sr=sr)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=sr)

    fig.savefig(image_file)
    plt.close(fig)

def create_pngs_from_wavs(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    dir = os.listdir(input_path)

    for i, file in enumerate(dir):
        input_file = os.path.join(input_path, file)
        output_file = os.path.join(output_path, file.replace('.wav', '.png'))
        audio_to_mel(input_file, output_file)

In [ ]:
def load_images_from_path(path, label):
    images = []
    labels = []
    filenames = []

    for language in os.listdir(path):
      language_path = os.path.join(path, language)
      for gender in os.listdir(language_path):
        gender_path = os.path.join(language_path, gender)
        for emotion in os.listdir(gender_path):
          emotion_path = os.path.join(gender_path, emotion)
          for img in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, img)
            img = image.load_img(img_path, target_size=(224, 224, 3))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
            filenames.append(os.path.basename(img_path).split('.')[0])

    return images, labels, filenames

def show_images(images):
    fig, axes = plt.subplots(1, len(images), figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)

x = []
y = []
files = []

In [ ]:
source_dir = '/content/gdrive/MyDrive/Multilingual Speech Dataset/'
output_dir = '/content/gdrive/MyDrive/Multilingual Speech Dataset Images/'

for folder in os.listdir(source_dir):
    filename = os.path.join(source_dir, folder)
    filename_1 = os.path.join(output_dir, folder)
    print("Filename:", filename)
    for gender in os.listdir(filename):
      gender_path = os.path.join(filename, gender)
      gender_path_1 = os.path.join(filename_1, gender)
      print("Gender Path:", gender_path)
      for emotion in os.listdir(gender_path):
        emotion_path = os.path.join(gender_path, emotion)
        emotion_path_1 =os.path.join(gender_path_1, emotion)
        print("Emotion Path:", emotion_path)
        for audio in os.listdir(emotion_path):
            if audio.endswith('.wav'):
                # Construct the full path to the source file
                source_path = os.path.join(emotion_path, audio)
                output_path = os.path.join(emotion_path_1, audio)

                # Create the destination directory if it doesn't exist
                create_pngs_from_wavs(emotion_path, emotion_path_1)

                print(f"Converted {filename} and saved to {output_path}")

print("Conversion complete.")

Filename: /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German
Gender Path: /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German/Female
Emotion Path: /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German/Female/Happy
Converted /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German and saved to /content/gdrive/MyDrive/Multilingual Speech Dataset Images/EmoDB - German/Female/Happy/08a01Fd.wav
Converted /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German and saved to /content/gdrive/MyDrive/Multilingual Speech Dataset Images/EmoDB - German/Female/Happy/09a01Fa.wav
Converted /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German and saved to /content/gdrive/MyDrive/Multilingual Speech Dataset Images/EmoDB - German/Female/Happy/13a01Fd.wav
Converted /content/gdrive/MyDrive/Multilingual Speech Dataset/EmoDB - German and saved to /content/gdrive/MyDrive/Multilingual Speech Dataset Images/EmoDB - German/Female/Happ

In [ ]:
drive_folder_path = '/content/gdrive/MyDrive/Multilingual Speech Dataset Images/'
languages = ['Angry', 'Disgust', 'Happy', 'Neutral', 'Sad']
label_encoder = LabelEncoder()

for language in languages:
    images, labels, filenames = load_images_from_path(drive_folder_path, language)
    show_images(images)
    x += images
    y += labels
    files += filenames

In [ ]:
# Convert string labels to numerical labels
y_numerical = label_encoder.fit_transform(y)

# Split the data(80% training, 20% test)
x_temp, x_test, y_temp, y_test, files_temp, files_test = train_test_split(x, y_numerical, files, stratify=y_numerical, test_size=0.2, random_state=0)

# Split the training data(60% training, 20% validation)
x_train, x_val, y_train, y_val, files_train, files_val = train_test_split(x_temp, y_temp, files_temp, stratify=y_temp, test_size=0.25, random_state=0)

# Normalize the images
x_train_norm = np.array(x_train) / 255
x_test_norm = np.array(x_test) / 255
x_val_norm = np.array(x_val) / 255

# One-hot encode the numerical labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)
y_val_encoded = to_categorical(y_val)

In [ ]:
from PIL import Image
import os

def save_images(images, labels, filenames, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

    for img_array, label, filename in zip(images, labels, filenames):
        img = Image.fromarray(img_array.astype('uint8'), 'RGB')
        file_path = os.path.join(directory, f"{filename}.png")
        img.save(file_path)

save_images(x_train, y_train, files_train, '/content/gdrive/MyDrive/Multilingual_train/')
save_images(x_test, y_test, files_test, '/content/gdrive/MyDrive/Multilingual_test/')
save_images(x_val, y_val, files_val, '/content/gdrive/MyDrive/Multilingual_validation/')

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow(x_train_norm, y_train_encoded, batch_size=32)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in the base ResNet model
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_save_path = '/content/gdrive/MyDrive/Multilingual_model.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=model_save_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1)


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(train_generator, steps_per_epoch=len(x_train_norm) / 32, epochs=50,
                    validation_data=(x_val_norm, y_val_encoded),
                    callbacks=[model_checkpoint_callback, early_stop, reduce_lr])


Epoch 1/50
53/52 [==============================] - ETA: 0s - loss: 1.8684 - accuracy: 0.1908
Epoch 1: val_accuracy did not improve from 0.23512
52/52 [==============================] - 38s 426ms/step - loss: 1.8684 - accuracy: 0.1908 - val_loss: 216784576.0000 - val_accuracy: 0.2004 - lr: 0.0100
Epoch 2/50
53/52 [==============================] - ETA: 0s - loss: 1.7657 - accuracy: 0.2075
Epoch 2: val_accuracy did not improve from 0.23512
52/52 [==============================] - 18s 342ms/step - loss: 1.7657 - accuracy: 0.2075 - val_loss: 176284.8750 - val_accuracy: 0.2004 - lr: 0.0100
Epoch 3/50
53/52 [==============================] - ETA: 0s - loss: 1.7236 - accuracy: 0.2194
Epoch 3: val_accuracy did not improve from 0.23512
52/52 [==============================] - 18s 343ms/step - loss: 1.7236 - accuracy: 0.2194 - val_loss: 6446.1396 - val_accuracy: 0.2004 - lr: 0.0100
Epoch 4/50
53/52 [==============================] - ETA: 0s - loss: 1.7237 - accuracy: 0.1932
Epoch 4: val_accurac